In [1]:
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
from sklearn.metrics import jaccard_score
import random
random.seed(10)
import pandas as pd
df = pd.read_excel (r"D:\work\Tasks\Data_13062022\955\FinalData_Train_Test\FinalDataArrange.xlsx",0, header=0)
df.shape

(955, 2010)

In [2]:
data = df.values
data.shape

(955, 2010)

In [10]:
X, y = data[:,1:2009], data[:, -1]
print(X.shape, y.shape)

(955, 2008) (955,)


In [4]:
X

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 1],
       ...,
       [1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 1, ..., 1, 0, 0]], dtype=object)

In [5]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,

In [11]:
from scipy.spatial import distance
def SphereExclusion_1(X,y,c):
    '''
    First center is selected as the one with maximum activity, 
    then centers are selected randomly
    ''' 
    random.seed(10)
    #Normalize the features using MinMaxScaler
    #scaler = MinMaxScaler()
    
    #X_norm = scaler.fit_transform(X)
    X_norm =X
    #Find cmpnd with maximum activity
    center_index = np.argmax(y)
    trainingset_indices = [center_index]
    K = X_norm.shape[1]
    N = X_norm.shape[0] 
    #Create radius
    R = c*(1/N)**(1/K)
    testset_indices = []
    #include points within the sphere to test set
    def select_test_set(X_norm, center_index, trainingset_indices, testset_indices):
        indices_removed = trainingset_indices + testset_indices
        center = X_norm[center_index, :]
        for i in range(X_norm.shape[0]):
            if(i not in indices_removed):
                if(distance.rogerstanimoto(X_norm[i,:],center) < R):
                    testset_indices.append(i)
        return(testset_indices)
    testset_indices = select_test_set(X_norm, center_index, trainingset_indices, testset_indices)
    X_updated_size = X_norm.shape[0]-(len(trainingset_indices)+len(testset_indices))
    while(X_updated_size > 0):
        indices_tobe_removed = trainingset_indices + testset_indices
        indices_to_select = []
        for i in range(X_norm.shape[0]):
            if(i not in indices_tobe_removed):
                indices_to_select.append(i)
        center_index = random.choice(indices_to_select)
        trainingset_indices.append(center_index)
        testset_indices = select_test_set(X_norm,center_index,trainingset_indices,testset_indices)
        X_updated_size = X_norm.shape[0]-(len(trainingset_indices)+len(testset_indices))
    
    return(trainingset_indices, testset_indices)

In [12]:
trainingset_indices,testset_indices = SphereExclusion_1(X,y,0.1)

In [13]:
len(testset_indices)

247

In [8]:
X_train_df=df.iloc[trainingset_indices]
X_test_df=df.iloc[testset_indices]

In [9]:
X_train_df.to_excel("D:/work/Tasks/Data_13062022/955/FinalData_Train_Test/SE/trainSet_SE_0.1.xlsx",sheet_name='train')
X_test_df.to_excel("D:/work/Tasks/Data_13062022/955/FinalData_Train_Test/SE/testSet_SE_0.1.xlsx",sheet_name='test')